In [1]:
import tensorflow as tf

from support.util import Config, Experiment

from trainable.models.vit import build_LAXNeXt_iterative_mean
from trainable.models.cnn import build_basic_cnn, build_EfficientNetB0, build_ICNN_DenseNet121, build_DenseNet121

from data.datasets.image_classification import cats_dogs, cifar10, imagenet, dot_dataset
from optimization.data_augmentation.ssda import custom_rand_augment_dset

from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from optimization.callbacks import EarlyStoppingDifference, EarlyStoppingOQ

from optimization.training_loops.supervised import keras_supervised, wandb_supervised
from optimization.schedules import BleedOut

"""
hardware_params must include:"

    'n_gpu': uint
    'n_cpu': uint
    'node': str
    'partition': str
    'time': str (we will just write this to the file)
    'memory': uint
    'results_dir': str
"""
hardware_params = {
    'name': 'hparam',
    'n_gpu': 2,
    'n_cpu': 16,
    'partition': 'ai2es',
    'nodelist': ['c830'],
    'time': '96:00:00',
    'memory': 16384,
    # The %04a is translated into a 4-digit number that encodes the SLURM_ARRAY_TASK_ID
    'stdout_path': '/ourdisk/hpc/ai2es/jroth/supercomputer/text_outputs/exp%01a_stdout_%A.txt',
    'stderr_path': '/ourdisk/hpc/ai2es/jroth/supercomputer/text_outputs/exp%01a_stderr_%A.txt',
    'email': 'jay.c.rothenberger@ou.edu',
    'dir': '/ourdisk/hpc/ai2es/jroth/AI2ES_DL/',
    'array': '[0-7%4]',
    'results_dir': 'ICNN'
}
"""
network_params must include:
    
    'network_fn': network building function
    'network_args': arguments to pass to network building function
        network_args must include:
            'lrate': float
    'hyperband': bool
"""
image_size = (256, 256, 3)

network_params = {
    'network_fn': build_ICNN_DenseNet121,
    'network_args': {
        'n_classes': 2,
        'conv_filters': 14,
        'conv_size': '[3]',
        'dense_layers': '[16]',
        'learning_rate': 1e-4,
        'image_size': image_size,
        'l1': None,
        'l2': 1e-4,
        'alpha': [2 ** (-6)],
        'beta': [0],
        'gamma': 0.0,
        'noise_level': 0.005,
        'depth': 4,
        'search_space': ['dropout'],
        'dropout': [0.05 * i for i in range(20)]
    },

    'hyperband': False
}

"""
experiment_params must include:
    
    'seed': random seed for computation
    'steps_per_epoch': uint
    'patience': uint
    'min_delta': float
    'epochs': uint
    'nogo': bool
"""

experiment_params = {
    'seed': 42,
    'steps_per_epoch': 196,
    'patience': 5,
    'min_delta': 0.0,
    'epochs': 32,
    'nogo': False,
    'train_metrics': [],
    'val_metrics': [],
    'project': 'ICNN',
    'entity': 'ai2es'
}
"""
dataset_params must include:
    'dset_fn': dataset loading function
    'dset_args': arguments for dataset loading function
    'cache': str or bool
    'batch': uint
    'prefetch': uint
    'shuffle': bool
    'augs': iterable of data augmentation functions
"""
dataset_params = {
    'dset_fn': cats_dogs,
    'dset_args': {
        'image_size': image_size[:-1],
        'path': '../data/'
    },
    'cache': True,
    'cache_to_lscratch': False,
    'batch': 64,
    'prefetch': tf.data.AUTOTUNE,
    'shuffle': True,
    'augs': []
}

"""
optimization_params must include:
    'callbacks':  iterable - callbacks for training
    'training_loop': callable - training loop to use for model weight update
"""

optimization_params = {
    'callbacks': [
        EarlyStopping(patience=experiment_params['patience'],
                        restore_best_weights=True,
                        min_delta=experiment_params['min_delta']),
        # LearningRateScheduler(BleedOut(network_params['network_args']['learning_rate'])),
        # LossWeightScheduler(loss_weight_schedule)
    ],
    'training_loop': keras_supervised
}

config = Config(hardware_params, network_params, dataset_params, experiment_params, optimization_params)

exp = Experiment(config)

# print(exp.params)
for i in range(5):
    exp.run_array(0)

# exp.enqueue()



2023-04-12 09:15:03.241674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/fagg/miniconda3/envs/tf_bleeding5/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Size of Hyperparameter Grid: 20
<class 'dict'>
{
	name: hparam
	n_gpu: 2
	n_cpu: 16
	partition: ai2es
	nodelist: ['c830']
	time: 96:00:00
	memory: 16384
	stdout_path: /ourdisk/hpc/ai2es/jroth/supercomputer/text_outputs/exp%01a_stdout_%A.txt
	stderr_path: /ourdisk/hpc/ai2es/jroth/supercomputer/text_outputs/exp%01a_stderr_%A.txt
	email: jay.c.rothenberger@ou.edu
	dir: /ourdisk/hpc/ai2es/jroth/AI2ES_DL/
	array: [0-7%4]
	results_dir: ICNN
	}
{
	seed: 42
	steps_per_epoch: 196
	patience: 5
	min_delta: 0.0
	epochs: 32
	nogo: False
	train_metrics: []
	val_metrics: []
	project: ICNN
	entity: ai2es
	}
{
	network_fn: <function build_ICNN_DenseNet121 at 0x2b33ffc8eef0>
	network_args: {
		hyperband: False
		dense_layers: [16]
		beta: 0
		n_classes: 2
		image_size: (256, 256, 3)
		conv_size: [3]
		depth: 4
		l2: 0.0001
		noise_level: 0.005
		l1: None
		learning_rate: 0.0001
		dropout: 0.0
		gamma: 0.0
		conv_filters: 14
		alpha: 0.015625
		}
	hyperband: False
	}
{
	dset_fn: <function cats_dogs at 0x

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

Corrupt JPEG data: 104 extraneous bytes before marker 0xd9
Corrupt JPEG data: 402 extraneous bytes before marker 0xd9
Corrupt JPEG data: 70 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2232 extraneous bytes before marker 0xd9
Corrupt JPEG data: 242 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1158 extraneous bytes before marker 0xd9
Corrupt JPEG data: 232 extraneous bytes before marker 0xd9
Corrupt JPEG data: 104 extraneous bytes before marker 0xd9
Corrupt JPEG data: 402 extraneous bytes before marker 0xd9
Corrupt JPEG data: 70 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2232 extraneous bytes before marker 0xd9
Corrupt JPEG data: 242 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1158 extraneous bytes before marker 0xd9
Corrupt JPEG data: 232 extraneous bytes before marker 0xd9
Corrupt JPEG data: 133 extraneous bytes before marker 0xd9

dense_categorical_accuracy,▁▁▁▁▄▅▅▆▆▇▇▇▇▇████
dense_loss,████▇▆▅▅▄▃▃▃▂▂▂▁▁▁
epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
loss,████▇▆▅▅▄▃▃▃▂▂▂▁▁▁
tf.__operators__.add_loss,▂▁▁▂▅▆▇▇▇████████▇
val_dense_categorical_accuracy,▁▁▁▁▅▅▆▇▆▇▇▇▇▇▇▇██
val_dense_loss,▆▆▆█▄▄▃▂▃▂▁▂▁▃▁▄▁▁
val_loss,███ ▆▅▄▃▃▂▁▂▁▄▁▆▁▁
val_tf.__operators__.add_loss,▁▁▁ █▃▄▄▃▄▄▅▄▅▄▆▅▄
GFLOPs,4.3158
dense_categorical_accuracy,0.95942


196 None
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 densenet121 (Functional)       (None, 8, 8, 1024)   7037504     ['input_4[0][0]']                
                                                                                                  
 activation_1 (Activation)      (None, 8, 8, 1024)   0           ['densenet121[0][0]']            
                                                                                                  
 conv_mask_2 (ConvMask)         ((None, 8, 8, 1024)  4160        ['activation_1[0][

2023-04-12 09:38:40.364506: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_INT64
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 16283
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\026TensorSliceDataset:628"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 256
        }
        dim {
          size: 256
        }
        dim {
          size: 3
        }
      }
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
 

AttributeError: 'str' object has no attribute 'set_model'

In [1]:
from tensorflow.keras.applications import ResNet50V2, EfficientNetB0
from tensorflow.keras.layers import Input
import tensorflow as tf

image_size = (244, 244, 3)
inputs = Input(image_size)

model = EfficientNetB0(input_tensor=inputs, include_top=True, weights='imagenet')

opt = tf.keras.optimizers.Nadam(learning_rate=1e-4,
                                beta_1=0.9, beta_2=0.999,
                                epsilon=None, decay=0.99)
opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)

accuracy = 'sparse_categorical_accuracy'
# compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=[accuracy])

2023-04-04 08:59:22.805624: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 08:59:23.017245: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-04 08:59:29.496119: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, re

In [2]:
from data.datasets.image_classification import cats_dogs, cifar10, imagenet

datasets = imagenet((244, 244, 3))

train = datasets['train'].batch(256)
val = datasets['val'].batch(256)

/home/fagg/miniconda3/envs/tf_bleeding5/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Found 50000 files belonging to 1000 classes.
Found 50000 files belonging to 1000 classes.
Found 1281167 files belonging to 1000 classes.
Found 1281167 files belonging to 1000 classes.


In [ ]:
model.summary()

In [3]:
model.evaluate(val)

2023-04-04 09:03:07.295503: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-04-04 09:03:10.346302: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Running ptxas --version returned 32512
2023-04-04 09:03:10.478156: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: ptxas exited with non-zero error code 32512, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


  1/196 [..............................] - ETA: 37:09 - loss: 0.6199 - sparse_categorical_accuracy: 0.8555

2023-04-04 09:03:10.733225: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


196/196 [==============================] - 433s 2s/step - loss: 0.9976 - sparse_categorical_accuracy: 0.7666


[0.9976070523262024, 0.7665600180625916]

In [ ]:
model.evaluate(train)

1028/5005 [=====>........................] - ETA: 2:23:49 - loss: 0.5368 - sparse_categorical_accuracy: 0.8851

In [4]:
from keras.utils import dataset_utils
def file_order(d):
    image_paths, labels, class_names = dataset_utils.index_directory(d, 'inferred', '.jpeg', shuffle=False)
    return image_paths

file_order('/scratch/jroth/2012/ILSVRC2012_val2/')

Found 50000 files belonging to 1000 classes.


['/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00000293.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00002138.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00003014.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00006697.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00007197.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00009111.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00009191.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00009346.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00009379.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00009396.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00010306.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSVRC2012_val_00011233.JPEG',
 '/scratch/jroth/2012/ILSVRC2012_val2/n01440764/ILSV

In [11]:
import hashlib,base58,binascii,ecdsa, codecs

ecdsaPrivateKey = ecdsa.SigningKey.from_secret_exponent(420, curve=ecdsa.SECP256k1)
print("ECDSA Private Key: ", ecdsaPrivateKey.to_string().hex())

ecdsaPublicKey = '04' +  ecdsaPrivateKey.get_verifying_key().to_string().hex()
print("ECDSA Public Key: ", ecdsaPublicKey)

hash256FromECDSAPublicKey = hashlib.sha256(binascii.unhexlify(ecdsaPublicKey)).hexdigest()
print("SHA256(ECDSA Public Key): ", hash256FromECDSAPublicKey)

ridemp160FromHash256 = hashlib.new('ripemd160', binascii.unhexlify(hash256FromECDSAPublicKey))
print("RIDEMP160(SHA256(ECDSA Public Key)): ", ridemp160FromHash256.hexdigest())

prependNetworkByte = '00' + ridemp160FromHash256.hexdigest()
print("Prepend Network Byte to RIDEMP160(SHA256(ECDSA Public Key)): ", prependNetworkByte)

h = prependNetworkByte
for x in range(1,3):
    h = hashlib.sha256(binascii.unhexlify(h)).hexdigest()
    print("\t|___>SHA256 #", x, " : ", h)
    
cheksum = h[:8]
print("Checksum(first 4 bytes): ", cheksum)

appendChecksum = prependNetworkByte + cheksum
print("Append Checksum to RIDEMP160(SHA256(ECDSA Public Key)): ", appendChecksum)

bitcoinAddress = base58.b58encode(binascii.unhexlify(appendChecksum))
print("Bitcoin Address: ", bitcoinAddress.decode('utf8'))

ECDSA Private Key:  00000000000000000000000000000000000000000000000000000000000001a4
ECDSA Public Key:  04cdaf553843b6c2410bfd83186bad43acacd81f2c129e2e9ba393dd3cc13daf9ba6c41db92ac40552af23295dbbaedf94947024ebadaded8ed5e04d1257cb1f59
SHA256(ECDSA Public Key):  67582dade20d4c29ca3d75aea61fbd1ac7c23f90a48a6b9107a45bbd9b05b09b
RIDEMP160(SHA256(ECDSA Public Key)):  9d5efee279808177d688c64f083ecef99c6591dd
Prepend Network Byte to RIDEMP160(SHA256(ECDSA Public Key)):  009d5efee279808177d688c64f083ecef99c6591dd
	|___>SHA256 # 1  :  3634c28fd8540c2d66b1e0af4641c693e856dd3acef33a2d863cf847284398f5
	|___>SHA256 # 2  :  2d46bffc890f308b64c183bcf2647f79258e8c3fcdc2a49c8f1fa4ba55a447ba
Checksum(first 4 bytes):  2d46bffc
Append Checksum to RIDEMP160(SHA256(ECDSA Public Key)):  009d5efee279808177d688c64f083ecef99c6591dd2d46bffc
Bitcoin Address:  1FM6tXmCajqRYcvK8wXN4KepUZ2aMyn11u


In [44]:
import numpy as np

def read_array_from_iostream(fp, index, shape, dtype=np.float32):
    itemsize = np.dtype(dtype).itemsize  # size is given in bytes
    blocksize = itemsize * np.prod(shape)
    # seek to the position of the array
    fp.seek(index * blocksize)
    # return read array resized to the correct shape
    return np.reshape(np.frombuffer(fp.read(blocksize), dtype=dtype), shape)
    

arr = np.zeros((64, 64, 3)).tobytes() # default float64 (8 bytes)
print(len(arr) == 64*64*3*8)

with open('bytearray', 'wb') as fp:
    fp.write(arr)
    fp.write(arr)

with open('bytearray', 'rb') as fp:
    print(read_array_from_iostream(fp, 1, (64, 64, 3), np.int64))
    

arr = np.arange(9)
print(arr.dtype)
arr = arr.tobytes()  # int64 -> bytes

with open('bytearray', 'wb') as fp:
    # write it twice
    fp.write(arr)
    fp.write(arr)

with open('bytearray', 'rb') as fp:
    # read the first array
    print(read_array_from_iostream(fp, 0, (3, 3), np.int64))
    # read the second array with the wrong shape
    print(read_array_from_iostream(fp, 1, (3, 2), np.int64))

True
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
int64
[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[6 7]
 [8 0]
 [1 2]]


In [37]:
import numpy as np

def read_array_from_iostream(fp, index, shape, dtype=np.float32):
    itemsize = np.dtype(dtype).itemsize  # size is given in bytes
    blocksize = itemsize * np.prod(shape)
    # seek to the position of the array
    fp.seek(index * blocksize)
    # return read array resized to the correct shape
    return np.reshape(np.frombuffer(fp.read(blocksize), dtype=dtype), shape)

def write_array_to_iostream(fp, array, index):
    shape = array.shape
    dtype = array.dtype
    itemsize = np.dtype(dtype).itemsize  # size is given in bytes
    blocksize = itemsize * np.prod(shape)
    # seek to the position of the array
    fp.seek(index * blocksize)
    # return number of written bytes
    written = fp.write(array.tobytes())
    fp.flush()
    return written

In [43]:
import torch
from time import perf_counter as time

fpw = open('cache', 'wb')
fpr = open('cache', 'rb')
start = time()
ones = torch.ones((64, 64, 3))
print(time() - start)
start = time()
np_ones = ones.numpy()
write_array_to_iostream(fpw, np_ones, 0)
print(time() - start)
start = time()
read = read_array_from_iostream(fpr, 0, (64, 64, 3), dtype=np_ones.dtype)
read = torch.tensor(read)
print(time() - start)
fpr.close()
fpw.close()

0.00012379512190818787
0.00031963130459189415
0.00021197879686951637


In [27]:
tensor = torch.ones((64, 64, 3))
blocksize = bytesio.getbuffer().nbytes # size is given in bytes


In [28]:
from io import BytesIO

def read_tensor_from_iostream(fp, index, shape):
    # seek to the position of the array
    fp.seek(index * blocksize)
    # return read array resized to the correct shape
    rax = torch.load(BytesIO(fp.read(blocksize)))
    return rax

def write_tensor_to_iostream(fp, index, tensor):
    # seek to the position of the array
    fp.seek(index * blocksize)
    torch.save(zeros, fp)
    fp.flush()

In [35]:
import torch
from time import perf_counter as time
fpw = open('cache', 'wb')
fpr = open('cache', 'rb')
start = time()
ones = torch.ones((64, 64, 3))
print(time() - start)
start = time()
read = write_tensor_to_iostream(fpw, 0, ones)
print(time() - start)
start = time()
read_tensor_from_iostream(fpr, 0, (64, 64, 3))
print(time() - start)
fpr.close()
fpw.close()

7.637310773134232e-05
0.0002915416844189167
0.00024714600294828415


In [21]:
import torch
from io import BytesIO
bytesio = BytesIO()
ones = torch.ones((64, 64, 3))
zeros = torch.zeros((64, 64, 3))
torch.save(ones, bytesio)
print(bytesio.getbuffer().nbytes)

torch.save(zeros, bytesio)
print(bytesio.getbuffer().nbytes)

bytesio.seek(0)
torch.save(zeros, bytesio)
print(bytesio.getbuffer().nbytes)

torch.load(BytesIO(bytesio.read(49899)))

49899
99798
99798


tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [22]:
read == torch.ones((64, 64, 3)).numpy()

array([[[False, False, False],
        [False, False, False],
        [False, False, False],
        ...,
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False],
        ...,
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False],
        ...,
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       ...,

       [[False, False, False],
        [False, False, False],
        [False, False, False],
        ...,
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False],
        ...,
        [False, False, False],
        [False, False, False],
        [False,

In [6]:
file_paths[:2]
labels = [j[0] for j in sorted([(i[0], i[1]) for i in np.load("/scratch/jroth/2012/ILSVRC2012_val_labels.npy", allow_pickle=True)], key=lambda k: k[-1])]
labels[0]

47

In [ ]:
from trainable.models.vit import *
from support.evaluations.image_classification import masking_evaluation

from support.util import load_most_recent_results

result_path = '../test_models/' #568735 (88% val acc)

results = load_most_recent_results(result_path)[0]


results.summary()
results.config.dataset_params['dset_args']['path'] = '../data/'
class_names = results.config.dataset_params['dset_fn'](**results.config.dataset_params['dset_args'])['class_names']
model_data = results.model_data
keras_model = model_data.get_model()
test_dset = results.config.dataset_params['dset_fn'](**results.config.dataset_params['dset_args'])['train']
test_dset = test_dset.batch(results.config.dataset_params['batch'])

for x, y in iter(test_dset):
    print(x.shape, y.shape)
    break

masking_evaluation(model_data, test_dset, class_names, 1)